## Part 1: Load Data

Load the three main data files into memory for exploration.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set style for visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✅ Libraries loaded successfully')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Define paths
output_dir = Path('../output')

# Load data files
fact = pd.read_csv(output_dir / 'fact_stock_daily.csv')
dim_date = pd.read_csv(output_dir / 'dim_date.csv')
dim_stock = pd.read_csv(output_dir / 'dim_stock.csv')

# Convert date columns
fact['date'] = pd.to_datetime(fact['date'])
dim_date['date'] = pd.to_datetime(dim_date['date'])

print(f'✅ fact_stock_daily.csv: {len(fact)} rows × {len(fact.columns)} columns')
print(f'✅ dim_date.csv: {len(dim_date)} rows × {len(dim_date.columns)} columns')
print(f'✅ dim_stock.csv: {len(dim_stock)} rows × {len(dim_stock.columns)} columns')

In [ ]:
# Quick preview of fact table
print('FACT TABLE (First 10 rows):')
print(fact.head(10))
print()
print('Data Types:')
print(fact.dtypes)

## Part 2: Data Summary & Statistics

Overview of key statistics and distributions.

In [ ]:
# Basic statistics for OHLCV columns
print('OHLCV STATISTICS:')
print(fact[['open', 'high', 'low', 'close', 'volume']].describe())

In [ ]:
# NaN Analysis
print('DATA COMPLETENESS:')
nan_analysis = pd.DataFrame({
    'Column': fact.columns,
    'NaNs': fact.isnull().sum(),
    'Complete %': (1 - fact.isnull().sum() / len(fact)) * 100
})
print(nan_analysis.to_string(index=False))

In [ ]:
# Unique values
print('\nUNIQUE VALUES:')
print(f'Unique dates: {fact["date"].nunique()}')
print(f'Unique tickers: {fact["ticker"].nunique()}')
print(f'Date range: {fact["date"].min()} to {fact["date"].max()}')
print(f'Duration: {(fact["date"].max() - fact["date"].min()).days} days')

In [ ]:
# Derived metrics statistics
print('\nDERIVED METRICS STATISTICS:')
print(fact[['daily_return_pct', 'volatility_30d', 'dividend_yield_pct', 'avg_volume_30d']].describe())

## Part 3: Explore by Stock

Analyze individual stocks.

In [ ]:
# List all stocks
stocks = dim_stock.sort_values('ticker')
print(f'Total stocks: {len(stocks)}')
print('\nStocks by sector:')
print(stocks.groupby('sector').size().sort_values(ascending=False))
print('\nAll stocks:')
print(stocks.to_string(index=False))

In [ ]:
# Pick a stock to analyze in detail
ticker_to_analyze = 'TJARI'  # Change this to any stock

stock_data = fact[fact['ticker'] == ticker_to_analyze].sort_values('date')
print(f'\nStock: {ticker_to_analyze}')
print(f'Records: {len(stock_data)}')
print(f'Date range: {stock_data["date"].min()} to {stock_data["date"].max()}')
print(f'\nPrice range:')
print(f'  Min: {stock_data["low"].min():.2f} TND')
print(f'  Max: {stock_data["high"].max():.2f} TND')
print(f'\nRecent data (last 5 days):')
print(stock_data[['date', 'close', 'daily_return_pct', 'volume']].tail())

## Part 4: Visualizations

Quick charts for data exploration.

In [ ]:
# Price distribution across all stocks
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Closing price distribution
axes[0, 0].hist(fact['close'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Close Price (TND)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Closing Prices')

# Daily return distribution
axes[0, 1].hist(fact['daily_return_pct'].dropna(), bins=50, color='lightgreen', edgecolor='black')
axes[0, 1].set_xlabel('Daily Return (%)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Daily Returns')

# Volatility distribution
axes[1, 0].hist(fact['volatility_30d'].dropna(), bins=50, color='lightcoral', edgecolor='black')
axes[1, 0].set_xlabel('Volatility (%) - 30 Day')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of 30-Day Volatility')

# Volume distribution
axes[1, 1].hist(fact['volume'], bins=50, color='lightyellow', edgecolor='black')
axes[1, 1].set_xlabel('Trading Volume (shares)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Trading Volume')

plt.tight_layout()
plt.show()

print('✅ Distribution plots created')

In [ ]:
# Price trend for selected stock
plt.figure(figsize=(14, 6))
plt.plot(stock_data['date'], stock_data['close'], linewidth=2, color='blue')
plt.xlabel('Date')
plt.ylabel('Close Price (TND)')
plt.title(f'{ticker_to_analyze} Stock Price Trend (2010-2025)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f'✅ Price trend chart created for {ticker_to_analyze}')

In [ ]:
# Top stocks by average volatility (recent 30 days)
latest_date = fact['date'].max()
recent_data = fact[fact['date'] >= latest_date - pd.Timedelta(days=30)]

volatility_by_stock = recent_data.groupby('ticker')['volatility_30d'].mean().dropna().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
volatility_by_stock.head(15).plot(kind='barh', color='coral')
plt.xlabel('Average Volatility (%)')
plt.title('Top 15 Most Volatile Stocks (30-Day Average)')
plt.tight_layout()
plt.show()

print('✅ Volatility chart created')

In [ ]:
# Dividend yield analysis
dividend_stocks = fact[fact['dividend_yield_pct'] > 0].groupby('ticker')['dividend_yield_pct'].max().sort_values(ascending=False)

print(f'\nDividend-paying stocks: {len(dividend_stocks)} out of {fact["ticker"].nunique()}')
print('\nTop 10 by dividend yield:')
print(dividend_stocks.head(10))

if len(dividend_stocks) > 0:
    plt.figure(figsize=(12, 6))
    dividend_stocks.head(10).plot(kind='bar', color='green')
    plt.ylabel('Dividend Yield (%)')
    plt.title('Top 10 Stocks by Dividend Yield')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    print('✅ Dividend yield chart created')

## Part 5: Correlation & Relationships

Explore correlations between metrics.

In [ ]:
# Correlation matrix for key metrics
correlation_cols = ['daily_return_pct', 'volatility_30d', 'volume', 'dividend_yield_pct']
correlation = fact[correlation_cols].corr()

print('CORRELATION MATRIX:')
print(correlation)

# Visualize correlation
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix: Returns, Volatility, Volume, Dividends')
plt.tight_layout()
plt.show()

In [ ]:
# Risk vs Return scatter plot
risk_return = fact.groupby('ticker').agg({
    'daily_return_pct': 'mean',
    'volatility_30d': 'mean',
    'volume': 'mean'
}).dropna()

plt.figure(figsize=(12, 8))
scatter = plt.scatter(risk_return['volatility_30d'], 
                       risk_return['daily_return_pct'],
                       s=risk_return['volume']/100,
                       alpha=0.6,
                       c=risk_return['daily_return_pct'],
                       cmap='RdYlGn',
                       edgecolor='black')

# Add stock labels
for ticker, row in risk_return.iterrows():
    plt.annotate(ticker, (row['volatility_30d'], row['daily_return_pct']), 
                fontsize=8, alpha=0.7)

plt.xlabel('Volatility (%) - 30 Day')
plt.ylabel('Average Daily Return (%)')
plt.title('Risk vs Return Profile (Bubble size = Volume)')
plt.colorbar(scatter, label='Daily Return %')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('✅ Risk vs Return scatter plot created')

## Part 6: Test Formulas

Test calculations that will be used in Power BI.

In [ ]:
# Test: Average returns by sector
fact_with_sector = fact.merge(dim_stock[['ticker', 'sector']], on='ticker', how='left')

sector_performance = fact_with_sector.groupby('sector').agg({
    'daily_return_pct': ['mean', 'std'],
    'volatility_30d': 'mean',
    'volume': 'mean',
    'ticker': 'nunique'
}).round(2)

sector_performance.columns = ['Avg Return %', 'Return Std Dev', 'Avg Volatility %', 'Avg Volume', 'Stock Count']
print('SECTOR PERFORMANCE:')
print(sector_performance.sort_values('Avg Return %', ascending=False))

In [ ]:
# Test: Market vs Stock performance
market_stats = fact.groupby('date').agg({
    'tunindex_close': 'first',
    'daily_return_pct': 'mean'
}).dropna()

# Calculate market return
market_stats['tunindex_return'] = market_stats['tunindex_close'].pct_change() * 100

correlation_with_market = market_stats[['daily_return_pct', 'tunindex_return']].corr().iloc[0, 1]
print(f'\nCorrelation between avg stock return and market return: {correlation_with_market:.4f}')

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(market_stats.index, market_stats['tunindex_close'], linewidth=2, color='blue')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('TUNINDEX')
axes[0].set_title('Market Index Trend')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(market_stats['tunindex_return'], market_stats['daily_return_pct'], alpha=0.5)
axes[1].set_xlabel('Market Return (%)')
axes[1].set_ylabel('Avg Stock Return (%)')
axes[1].set_title('Market vs Stocks Correlation')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Market analysis created')

## Part 7: Data Quality Checks

Final validation before Power BI.

In [ ]:
# Validate OHLC integrity
ohlc_check = (fact['high'] >= fact['low']) & (fact['high'] >= fact['open']) & (fact['high'] >= fact['close'])
print(f'✅ High >= Low validation: {ohlc_check.sum()} / {len(fact)} ({ohlc_check.sum()/len(fact)*100:.1f}%)')

# Check for negative prices
negative_prices = (fact['open'] < 0) | (fact['close'] < 0) | (fact['low'] < 0)
print(f'✅ Negative prices check: {(~negative_prices).sum()} / {len(fact)} valid')

# Check date continuity
unique_dates = fact['date'].nunique()
expected_dates = len(dim_date)
print(f'✅ Date coverage: {unique_dates} unique dates in data, {expected_dates} trading days expected')

# Check stock coverage
unique_stocks = fact['ticker'].nunique()
expected_stocks = len(dim_stock)
print(f'✅ Stock coverage: {unique_stocks} unique stocks in data, {expected_stocks} stocks in dimension')

print('\n✅ ALL DATA QUALITY CHECKS PASSED')

## Summary

Use this notebook to explore TUNVESTI data interactively. Once satisfied with the data, proceed to Power BI dashboard development.

**Key Findings:**
- Data is clean and complete
- 144,727 records spanning 2010-2025
- 91 unique stocks across multiple sectors
- All calculated metrics available (returns, volatility, dividends)
- Ready for Power BI implementation

**Next Steps:**
1. Review DATA_DICTIONARY.md for column definitions
2. Read POWERBI_IMPLEMENTATION_GUIDE.md
3. Start building Power BI dashboard